In [1]:
from __future__ import division, print_function
%matplotlib inline

In [2]:
import sympy
from sympy import symbols, sin, cos, sqrt, diag
from sympy import Matrix, BlockMatrix, eye, ones, zeros
from sympy.physics.mechanics import *
from IPython.display import display
sympy.init_printing(use_latex='mathjax')

import numpy as np



# Quaternion Math

In [3]:
def expq(n):
    n *= 0.5
    nNorm = n.norm()
    qn = Matrix([cos(nNorm),n/nNorm*sin(nNorm)])
    return qn

def quat2dcm(q):
    """
    Convert quaternion to DCM
    """
    
    # Extract components
    w = q[0]
    x = q[1]
    y = q[2]
    z = q[3]
    
    # Reduce repeated calculations
    ww = w*w
    xx = x*x
    yy = y*y
    zz = z*z  
    wx = w*x
    wy = w*y
    wz = w*z
    xy = x*y
    xz = x*z
    yz = y*z
    
    # Build Direction Cosine Matrix (DCM)   
    dcm = Matrix([
        [ww + xx - yy - zz,       2*(xy - wz),       2*(xz + wy)],
        [      2*(xy + wz), ww - xx + yy - zz,       2*(yz - wx)],
        [      2*(xz - wy),       2*(yz + wx), ww - xx - yy + zz]
    ])
    return dcm

def dcm2quat(dcm):
    """
    Determine quaternion corresponding to dcm using
    the stanley method. 
    
    Flips sign to always return shortest path quaterion
    so w >= 0
    
    Converts the 3x3 DCM into the quaterion where the 
    first component is the real part
    """
    
    tr = Matrix.trace(dcm)
    
    w = 0.25*(1+tr)
    x = 0.25*(1+2*dcm[0,0]-tr)
    y = 0.25*(1+2*dcm[1,1]-tr)
    z = 0.25*(1+2*dcm[2,2]-tr)
    
    #kMax = np.argmax([w,x,y,z])
    
    kMax = 0
    
    if kMax == 0:
        w = sqrt(w)
        x = 0.25*(dcm[1,2]-dcm[2,1])/w
        y = 0.25*(dcm[2,0]-dcm[0,2])/w
        z = 0.25*(dcm[0,1]-dcm[1,0])/w
    
    elif kMax == 1:
        x = sqrt(x)
        w = 0.25*(dcm[1,2]-dcm[2,1])/x
        if w<0:
            x = -x
            w = -w
        y = 0.25*(dcm[0,1]+dcm[1,0])/x
        z = 0.25*(dcm[2,0]+dcm[0,2])/x
        
    elif kMax == 2:
        y = sqrt(y)
        w = 0.25*(dcm[2,0]-dcm[0,2])/y
        if w<0:
            y = -y
            w = -w
        x = 0.25*(dcm[0,1]+dcm[1,0])/y
        z = 0.25*(dcm[1,2]+dcm[2,1])/y
        
    elif kMax == 3:
        z = sqrt(z)
        w = 0.25*(dcm[0,1]-dcm[1,0])/z
        if w<0:
            z = -z
            w = -w
        x = 0.25*(dcm[2,0]+dcm[0,2])/z
        y = 0.25*(dcm[1,2]+dcm[2,1])/z
        
    q = Matrix([w,x,y,z])
    
    return q

def skew3(v):
    vx,vy,vz = v
    out = Matrix([[  0, -vz,   vy],
                  [ vz,   0,  -vx],
                  [-vy,  vx,    0]])
    return out

def skew4Left(v):
    if len(v)==3:
        v = Matrix.vstack(zeros(1),v)
    w,x,y,z = v
    out = Matrix([
            [w, -x, -y, -z],
            [x,  w, -z,  y],
            [y,  z,  w, -x],
            [z, -y,  x,  w],
        ])        
    return out

def skew4Right(v):
    if len(v)==3:
        v = Matrix.vstack(zeros(1),v)
    w,x,y,z = v
    out = Matrix([
            [w, -x, -y, -z],
            [x,  w,  z, -y],
            [y, -z,  w,  x],
            [z,  y, -x,  w],
        ])      
    return out


def quatConj(q):
    q_out = Matrix(q[:])
    q_out = q_out.T*sympy.diag(1,-1,-1,-1)
    q_out = q_out.T

    return q_out

def qRot(q,v):
    qPrime = quatConj(q)
    v = Matrix.vstack(zeros(1),v)
    vout = skew4Left(q)*skew4Right(qPrime)*v
    return Matrix(vout[1:])


def dRotdq(q,v):
    qw,qx,qy,qz = q
    vx,vy,vz = v

    dRdq = Matrix([
        [2*qw*vx + 2*qy*vz - 2*qz*vy,  2*qx*vx + 2*qy*vy + 2*qz*vz,  2*qw*vz + 2*qx*vy - 2*qy*vx, -2*qw*vy + 2*qx*vz - 2*qz*vx],
        [2*qw*vy - 2*qx*vz + 2*qz*vx, -2*qw*vz - 2*qx*vy + 2*qy*vx,  2*qx*vx + 2*qy*vy + 2*qz*vz,  2*qw*vx + 2*qy*vz - 2*qz*vy],
        [2*qw*vz + 2*qx*vy - 2*qy*vx,  2*qw*vy - 2*qx*vz + 2*qz*vx, -2*qw*vx - 2*qy*vz + 2*qz*vy,  2*qx*vx + 2*qy*vy + 2*qz*vz]
    ])

    return dRdq

def dVdq(q,v):
    qw,qx,qy,qz = q
    vx,vy,vz = v
    
    dv = Matrix([
        [ 2*vx*qw + 2*vy*qz - 2*vz*qy, 2*vx*qx + 2*vy*qy + 2*vz*qz, -2*vx*qy + 2*vy*qx - 2*vz*qw, -2*vx*qz + 2*vy*qw + 2*vz*qx],
        [-2*vx*qz + 2*vy*qw + 2*vz*qx, 2*vx*qy - 2*vy*qx + 2*vz*qw,  2*vx*qx + 2*vy*qy + 2*vz*qz, -2*vx*qw - 2*vy*qz + 2*vz*qy],
        [ 2*vx*qy - 2*vy*qx + 2*vz*qw, 2*vx*qz - 2*vy*qw - 2*vz*qx,  2*vx*qw + 2*vy*qz - 2*vz*qy,  2*vx*qx + 2*vy*qy + 2*vz*qz],
    ])
    
    return dv


# Setup

In [4]:
# Time step
t = symbols('T')

# States
qw,qx,qy,qz = symbols('q_w q_x q_y q_z')
rx,ry,rz = symbols('r_x r_y r_z') 
vx,vy,vz = symbols('v_x v_y v_z')
abx,aby,abz = symbols('a_bx a_by a_bz')
gbx,gby,gbz = symbols('g_bx g_by g_bz')

r_L = Matrix([rx,ry,rz])
v_L = Matrix([vx,vy,vz])
q_toLfromB = Matrix([qw,qx,qy,qz])
gyroBias = Matrix([gbx,gby,gbz])
accelBias = Matrix([abx,aby,abz])

# Measurements
ax,ay,az = symbols('a_x a_y a_z') # measured
mx,my,mz = symbols('m_x m_y m_z') # measured 
wx,wy,wz = symbols('w_x w_y w_z') # measured

accelMeas = Matrix([ax,ay,az])
magMeas = Matrix([mx,my,mz]) 
wMeas = Matrix([wx,wy,wz])

# Nav vectors
g = symbols('g')
bx,by,bz = symbols('b_x b_y b_z') 

g_L = Matrix([0,0,g])
B_L = Matrix([mx,0,0])

# Measurment Uncertanties
eGyro, eAccel, eMag, eGPS = symbols('eGyro eAccel eMag eGPS')
ewx,ewy,ewz = symbols('e_wx e_wy e_wz')
eax,eay,eaz = symbols('e_ax e_ay e_az')
emx,emy,emz = symbols('e_mx e_my e_mz')

ew = Matrix([ewx,ewy,ewz])
ea = Matrix([eax,eay,eaz])
em = Matrix([emx,emy,emz])

eVec = Matrix([
    ew,
    ea,
    em,
])

esub0 = {ewx:0,ewy:0,ewz:0,
         eax:0,eay:0,eaz:0,
         emx:0,emy:0,emz:0}

#erx,ery,erz = symbols('e_wx e_wy w_wz')
#evx,evy,evz = symbols('e_ax e_ay w_az')
#ewx,ewy,ewz = symbols('e_wx e_wy w_wz')

#er = Matrix([erx,ery,erz])
#ev = Matrix([evx,evy,evz])
#ew = Matrix([ewx,ewy,ewz])

#eVec = Matrix([
#    er,
#    ev,
#    ew,
#])

#esub0 = {erx:0,ery:0,erz:0,
#         evx:0,evy:0,evz:0,
#         ewx:0,ewy:0,ewz:0}

wax,way,waz = symbols('w_ax w_ay w_az') 
wa = symbols('w_a')
ww = symbols('w_w')
waB = symbols('w_aB')
wgB = symbols('w_gB')


# Validate quaternion math

In [5]:
print('dcm')
display( quat2dcm(q_toLfromB) )
print('[q]L')
display( skew4Left(q_toLfromB) )
print('[q]R')
display( skew4Right(q_toLfromB) )
print('[wb]R')
display( skew4Right(wMeas) )
r_B = Matrix([rx,ry,rz])
print('d/dq R*r_B')
display(dRotdq(q_toLfromB, r_B))



dcm


⎡   2     2      2      2                                                    ⎤
⎢q_w  + qₓ  - q_y  - q_z    -2⋅q_w⋅q_z + 2⋅qₓ⋅q_y      2⋅q_w⋅q_y + 2⋅qₓ⋅q_z  ⎥
⎢                                                                            ⎥
⎢                             2     2      2      2                          ⎥
⎢  2⋅q_w⋅q_z + 2⋅qₓ⋅q_y    q_w  - qₓ  + q_y  - q_z    -2⋅q_w⋅qₓ + 2⋅q_y⋅q_z  ⎥
⎢                                                                            ⎥
⎢                                                       2     2      2      2⎥
⎣ -2⋅q_w⋅q_y + 2⋅qₓ⋅q_z      2⋅q_w⋅qₓ + 2⋅q_y⋅q_z    q_w  - qₓ  - q_y  + q_z ⎦

[q]L


⎡q_w  -qₓ   -q_y  -q_z⎤
⎢                     ⎥
⎢qₓ   q_w   -q_z  q_y ⎥
⎢                     ⎥
⎢q_y  q_z   q_w   -qₓ ⎥
⎢                     ⎥
⎣q_z  -q_y   qₓ   q_w ⎦

[q]R


⎡q_w  -qₓ   -q_y  -q_z⎤
⎢                     ⎥
⎢qₓ   q_w   q_z   -q_y⎥
⎢                     ⎥
⎢q_y  -q_z  q_w    qₓ ⎥
⎢                     ⎥
⎣q_z  q_y   -qₓ   q_w ⎦

[wb]R


⎡ 0   -wₓ   -w_y  -w_z⎤
⎢                     ⎥
⎢wₓ    0    w_z   -w_y⎥
⎢                     ⎥
⎢w_y  -w_z   0     wₓ ⎥
⎢                     ⎥
⎣w_z  w_y   -wₓ    0  ⎦

d/dq R*r_B


⎡2⋅q_w⋅rₓ + 2⋅q_y⋅r_z - 2⋅q_z⋅r_y  2⋅qₓ⋅rₓ + 2⋅q_y⋅r_y + 2⋅q_z⋅r_z    2⋅q_w⋅r_
⎢                                                                             
⎢2⋅q_w⋅r_y - 2⋅qₓ⋅r_z + 2⋅q_z⋅rₓ   -2⋅q_w⋅r_z - 2⋅qₓ⋅r_y + 2⋅q_y⋅rₓ   2⋅qₓ⋅rₓ 
⎢                                                                             
⎣2⋅q_w⋅r_z + 2⋅qₓ⋅r_y - 2⋅q_y⋅rₓ   2⋅q_w⋅r_y - 2⋅qₓ⋅r_z + 2⋅q_z⋅rₓ   -2⋅q_w⋅rₓ

z + 2⋅qₓ⋅r_y - 2⋅q_y⋅rₓ   -2⋅q_w⋅r_y + 2⋅qₓ⋅r_z - 2⋅q_z⋅rₓ⎤
                                                          ⎥
+ 2⋅q_y⋅r_y + 2⋅q_z⋅r_z   2⋅q_w⋅rₓ + 2⋅q_y⋅r_z - 2⋅q_z⋅r_y⎥
                                                          ⎥
 - 2⋅q_y⋅r_z + 2⋅q_z⋅r_y  2⋅qₓ⋅rₓ + 2⋅q_y⋅r_y + 2⋅q_z⋅r_z ⎦

# Extended Kalman Filter Equations

## State Equations

For the Linear Kalman filter we have the following for the process and measurment models:

$$\begin{aligned}\dot{\mathbf x} &= \mathbf{Ax} + w_x\\
\mathbf z &= \mathbf{Hx} + w_z
\end{aligned}$$

Where $\mathbf A$ is the systems dynamic matrix. Using the state space methodsthese equations can be transformed into:
$$\begin{aligned}\bar{\mathbf x} &= \mathbf{Fx} \\
\mathbf z &= \mathbf{Hx}
\end{aligned}$$

For the nonlinear model the linear expression $\mathbf{Fx} + \mathbf{Bu}$ is replaced by a nonlinear function $f(\mathbf x, \mathbf u)$, and the linear expression $\mathbf{Hx}$ is replaced by a nonlinear function $h(\mathbf x)$:

$$\begin{aligned}\dot{\mathbf x} &= f(\mathbf x, \mathbf u) + w_x\\
\mathbf z &= h(\mathbf x) + w_z
\end{aligned}$$

The EKF doesn't alter the Kalman Filter's linear equations. Instead, it *linearizes* the nonlinear equations at the point of the current estimate. The system is linearized by using the jacobian.

$$
\begin{aligned}
\mathbf F 
&= {\frac{\partial{f(\mathbf x_t, \mathbf u_t)}}{\partial{\mathbf x}}}\biggr|_{{\mathbf x_t},{\mathbf u_t}} \\
\mathbf H &= \frac{\partial{h(\bar{\mathbf x}_t)}}{\partial{\bar{\mathbf x}}}\biggr|_{\bar{\mathbf x}_t} 
\end{aligned}
$$

 
## Summary of Linear and Nonlinear Kalman Filter Equations


$$\begin{array}{l|l}
\text{Linear Kalman filter} & \text{EKF} \\
\hline 
& \boxed{\mathbf F = {\frac{\partial{f(\mathbf x_t, \mathbf u_t)}}{\partial{\mathbf x}}}\biggr|_{{\mathbf x_t},{\mathbf u_t}}} \\
\mathbf{\bar x} = \mathbf{Fx} + \mathbf{Bu} & \boxed{\mathbf{\bar x} = f(\mathbf x, \mathbf u)}  \\
\mathbf{\bar P} = \mathbf{FPF}^\mathsf{T}+\mathbf Q  & \mathbf{\bar P} = \mathbf{FPF}^\mathsf{T}+\mathbf Q \\
\hline
& \boxed{\mathbf H = \frac{\partial{h(\bar{\mathbf x}_t)}}{\partial{\bar{\mathbf x}}}\biggr|_{\bar{\mathbf x}_t}} \\
\textbf{y} = \mathbf z - \mathbf{H \bar{x}} & \textbf{y} = \mathbf z - \boxed{h(\bar{x})}\\
\mathbf{K} = \mathbf{\bar{P}H}^\mathsf{T} (\mathbf{H\bar{P}H}^\mathsf{T} + \mathbf R)^{-1} & \mathbf{K} = \mathbf{\bar{P}H}^\mathsf{T} (\mathbf{H\bar{P}H}^\mathsf{T} + \mathbf R)^{-1} \\
\mathbf x=\mathbf{\bar{x}} +\mathbf{K\textbf{y}} & \mathbf x=\mathbf{\bar{x}} +\mathbf{K\textbf{y}} \\
\mathbf P= (\mathbf{I}-\mathbf{KH})\mathbf{\bar{P}} & \mathbf P= (\mathbf{I}-\mathbf{KH})\mathbf{\bar{P}}
\end{array}$$





## Measurment Model 

The measurment model jacobian should be (N_measurments, N_states)

$$
\mathbf H = \frac{\partial{h(\bar{\mathbf x})}}{\partial{\bar{\mathbf x}}}\biggr|_{\bar{\mathbf x}_t} = 
\begin{bmatrix}
\frac{\partial h_1}{\partial x_1} & \frac{\partial h_1}{\partial x_2} &\dots \\
\frac{\partial h_2}{\partial x_1} & \frac{\partial h_2}{\partial x_2} &\dots \\
\vdots & \vdots
\end{bmatrix}
$$



# Quadcopter Model

Here we mode the system dynamics using a constant acceleration model where the inertial acceleration acts as the input to the inertial position and velocity functions. We also represent the body attitude using the quaternion representation.

The measurments consist of an imu+mag and gps. Together this provides body acceleration, body rates, estimated gravity vector, estimated north vector, inertial position (and possibly inertial velocity).

For now, the accel and gyro bias states will be excluded.

## State and Measurment Models

Constant acceleration model with acceleration as input and position sensor (GPS) measurements


In [6]:
# State vector
X = Matrix([
    r_L,
    v_L,
    q_toLfromB,
])

# State transition model (const accel & angAccel)
a_B = accelMeas
a_L  = qRot(q_toLfromB, a_B) - g_L

fx_r = r_L + v_L*t + 1/2*a_L*t**2
fx_v = v_L + a_L*t

wb = wMeas
fx_q = (eye(4) + t/2*skew4Right(wb))*q_toLfromB

#expq_wt = expq(wMeas*t)
#fx_q = skew4Left(q_toLfromB) * expq_wt

fx = Matrix([
    fx_r,
    fx_v,
    fx_q
])

# Measurment Model (estimates measurment from states)
hx_gps = r_L # used for position
hx_accel = -qRot(quatConj(q_toLfromB), g_L) # used for attitude
hx_mag   =  qRot(quatConj(q_toLfromB), B_L) # used for attitude

hx = Matrix([
    hx_gps,
    hx_accel,
    hx_mag,
])


# Display
print('State vector (X)')
display(X,X.shape)
print('State transition model (fx)')
display(fx,fx.shape)
print('Measurment function (hx)')
display(hx,hx.shape)


State vector (X)


⎡rₓ ⎤
⎢   ⎥
⎢r_y⎥
⎢   ⎥
⎢r_z⎥
⎢   ⎥
⎢vₓ ⎥
⎢   ⎥
⎢v_y⎥
⎢   ⎥
⎢v_z⎥
⎢   ⎥
⎢q_w⎥
⎢   ⎥
⎢qₓ ⎥
⎢   ⎥
⎢q_y⎥
⎢   ⎥
⎣q_z⎦

(10, 1)

State transition model (fx)


⎡      2 ⎛       ⎛   2     2      2      2⎞                                   
⎢     T ⋅⎝0.5⋅aₓ⋅⎝q_w  + qₓ  - q_y  - q_z ⎠ + 0.5⋅a_y⋅(-2⋅q_w⋅q_z + 2⋅qₓ⋅q_y) 
⎢                                                                             
⎢     2 ⎛                                        ⎛   2     2      2      2⎞   
⎢    T ⋅⎝0.5⋅aₓ⋅(2⋅q_w⋅q_z + 2⋅qₓ⋅q_y) + 0.5⋅a_y⋅⎝q_w  - qₓ  + q_y  - q_z ⎠ + 
⎢                                                                             
⎢ 2 ⎛                                                                         
⎢T ⋅⎝0.5⋅aₓ⋅(-2⋅q_w⋅q_y + 2⋅qₓ⋅q_z) + 0.5⋅a_y⋅(2⋅q_w⋅qₓ + 2⋅q_y⋅q_z) + 0.5⋅a_z
⎢                                                                             
⎢                 ⎛   ⎛   2     2      2      2⎞                              
⎢               T⋅⎝aₓ⋅⎝q_w  + qₓ  - q_y  - q_z ⎠ + a_y⋅(-2⋅q_w⋅q_z + 2⋅qₓ⋅q_y)
⎢                                                                             
⎢                ⎛                                ⎛ 

(10, 1)

Measurment function (hx)


⎡             rₓ              ⎤
⎢                             ⎥
⎢             r_y             ⎥
⎢                             ⎥
⎢             r_z             ⎥
⎢                             ⎥
⎢ -g⋅(-2⋅q_w⋅q_y + 2⋅qₓ⋅q_z)  ⎥
⎢                             ⎥
⎢  -g⋅(2⋅q_w⋅qₓ + 2⋅q_y⋅q_z)  ⎥
⎢                             ⎥
⎢   ⎛   2     2      2      2⎞⎥
⎢-g⋅⎝q_w  - qₓ  - q_y  + q_z ⎠⎥
⎢                             ⎥
⎢   ⎛   2     2      2      2⎞⎥
⎢mₓ⋅⎝q_w  + qₓ  - q_y  - q_z ⎠⎥
⎢                             ⎥
⎢ mₓ⋅(-2⋅q_w⋅q_z + 2⋅qₓ⋅q_y)  ⎥
⎢                             ⎥
⎣  mₓ⋅(2⋅q_w⋅q_y + 2⋅qₓ⋅q_z)  ⎦

(9, 1)

## State and Measurment Jacobians

Determine the jacobian of the state and measurment models to implement the EKF for nonlinear systems.


In [7]:
# State model jacobian
dfdx = fx.jacobian(X)

# Measurment model jacobian
dhdx = hx.jacobian(X)

# G (dfde)
#dfde = fx.jacobian(eVec)

# Display
print('State model jacobian (dfdx)')
display(dfdx,dfdx.shape)
print('Measurment model jacobian (dhdx)')
display(dhdx,dhdx.shape)


State model jacobian (dfdx)


⎡                   2                                        2                
⎢1  0  0  T  0  0  T ⋅(1.0⋅aₓ⋅q_w - a_y⋅q_z + 1.0⋅a_z⋅q_y)  T ⋅(1.0⋅aₓ⋅qₓ + 1.
⎢                                                                             
⎢                   2                                            2            
⎢0  1  0  0  T  0  T ⋅(1.0⋅aₓ⋅q_z + 1.0⋅a_y⋅q_w - a_z⋅qₓ)       T ⋅(1.0⋅aₓ⋅q_y
⎢                                                                             
⎢                   2                                          2              
⎢0  0  1  0  0  T  T ⋅(-aₓ⋅q_y + 1.0⋅a_y⋅qₓ + 1.0⋅a_z⋅q_w)    T ⋅(1.0⋅aₓ⋅q_z +
⎢                                                                             
⎢0  0  0  1  0  0   T⋅(2⋅aₓ⋅q_w - 2⋅a_y⋅q_z + 2⋅a_z⋅q_y)       T⋅(2⋅aₓ⋅qₓ + 2⋅
⎢                                                                             
⎢0  0  0  0  1  0    T⋅(2⋅aₓ⋅q_z + 2⋅a_y⋅q_w - 2⋅a_z⋅qₓ)       T⋅(2⋅aₓ⋅q_y - 2
⎢                                                   

(10, 10)

Measurment model jacobian (dhdx)


⎡1  0  0  0  0  0      0         0          0          0    ⎤
⎢                                                           ⎥
⎢0  1  0  0  0  0      0         0          0          0    ⎥
⎢                                                           ⎥
⎢0  0  1  0  0  0      0         0          0          0    ⎥
⎢                                                           ⎥
⎢0  0  0  0  0  0   2⋅g⋅q_y   -2⋅g⋅q_z   2⋅g⋅q_w    -2⋅g⋅qₓ ⎥
⎢                                                           ⎥
⎢0  0  0  0  0  0   -2⋅g⋅qₓ   -2⋅g⋅q_w  -2⋅g⋅q_z   -2⋅g⋅q_y ⎥
⎢                                                           ⎥
⎢0  0  0  0  0  0  -2⋅g⋅q_w    2⋅g⋅qₓ    2⋅g⋅q_y   -2⋅g⋅q_z ⎥
⎢                                                           ⎥
⎢0  0  0  0  0  0  2⋅mₓ⋅q_w   2⋅mₓ⋅qₓ   -2⋅mₓ⋅q_y  -2⋅mₓ⋅q_z⎥
⎢                                                           ⎥
⎢0  0  0  0  0  0  -2⋅mₓ⋅q_z  2⋅mₓ⋅q_y   2⋅mₓ⋅qₓ   -2⋅mₓ⋅q_w⎥
⎢                                                           ⎥
⎣0  0  0

(9, 10)

In [8]:
# State model jacobian (analytic solution)
dfdx_q_1stOrderApprox = eye(4) + t/2*skew4Right(wb)
dfdx_q = skew4Right(expq(0.5*t/2*wb))

dfdx = Matrix(BlockMatrix([
    [  eye(3),     t*eye(3), 0.5*t**2*dRotdq(q_toLfromB, a_B)],
    [zeros(3),       eye(3),        t*dRotdq(q_toLfromB, a_B)],
    [zeros(4,3), zeros(4,3),                           dfdx_q],

]))

# Meaurment jacobian (analytical solution)
dhdx = Matrix(BlockMatrix([
    [  eye(3), zeros(3),             zeros(3,4)],
    [zeros(3), zeros(3), dVdq(q_toLfromB, -g_L)],
    [zeros(3), zeros(3), dVdq(q_toLfromB,  B_L)],
]))

# Display
print('State model jacobian (dfdx)')
display(dfdx,dfdx.shape)
print('Measurment model jacobian (dhdx)')
display(dhdx,dhdx.shape)

State model jacobian (dfdx)


⎡                              2                                              
⎢1  0  0  T  0  0         0.5⋅T ⋅(2⋅aₓ⋅q_w - 2⋅a_y⋅q_z + 2⋅a_z⋅q_y)           
⎢                                                                             
⎢                              2                                              
⎢0  1  0  0  T  0         0.5⋅T ⋅(2⋅aₓ⋅q_z + 2⋅a_y⋅q_w - 2⋅a_z⋅qₓ)            
⎢                                                                             
⎢                              2                                              
⎢0  0  1  0  0  T         0.5⋅T ⋅(-2⋅aₓ⋅q_y + 2⋅a_y⋅qₓ + 2⋅a_z⋅q_w)           
⎢                                                                             
⎢0  0  0  1  0  0           T⋅(2⋅aₓ⋅q_w - 2⋅a_y⋅q_z + 2⋅a_z⋅q_y)              
⎢                                                                             
⎢0  0  0  0  1  0            T⋅(2⋅aₓ⋅q_z + 2⋅a_y⋅q_w - 2⋅a_z⋅qₓ)              
⎢                                                   

(10, 10)

Measurment model jacobian (dhdx)


⎡1  0  0  0  0  0      0         0          0          0    ⎤
⎢                                                           ⎥
⎢0  1  0  0  0  0      0         0          0          0    ⎥
⎢                                                           ⎥
⎢0  0  1  0  0  0      0         0          0          0    ⎥
⎢                                                           ⎥
⎢0  0  0  0  0  0   2⋅g⋅q_y   -2⋅g⋅q_z   2⋅g⋅q_w    -2⋅g⋅qₓ ⎥
⎢                                                           ⎥
⎢0  0  0  0  0  0   -2⋅g⋅qₓ   -2⋅g⋅q_w  -2⋅g⋅q_z   -2⋅g⋅q_y ⎥
⎢                                                           ⎥
⎢0  0  0  0  0  0  -2⋅g⋅q_w    2⋅g⋅qₓ    2⋅g⋅q_y   -2⋅g⋅q_z ⎥
⎢                                                           ⎥
⎢0  0  0  0  0  0  2⋅mₓ⋅q_w   2⋅mₓ⋅qₓ   -2⋅mₓ⋅q_y  -2⋅mₓ⋅q_z⎥
⎢                                                           ⎥
⎢0  0  0  0  0  0  -2⋅mₓ⋅q_z  2⋅mₓ⋅q_y   2⋅mₓ⋅qₓ   -2⋅mₓ⋅q_w⎥
⎢                                                           ⎥
⎣0  0  0

(9, 10)

## Process Noise Covariance $Q$

Unmodeled disturbances (wind) can cause a force to act on the body. This results in an acceleration disturbance and nees to be moddeled with the process noise covariance matrix $Q$.

$$Q = \begin{bmatrix}
    \sigma_{x}^2 & 0 & 0 & \sigma_{x \dot x} & 0 & 0\\
    0 & \sigma_{y}^2 & 0 & 0 & \sigma_{y \dot y} & 0 \\
    0 & 0 & \sigma_{z}^2 & 0 & 0 & \sigma_{z \dot z}\\
    \sigma_{\dot x x} & 0 & 0 & \sigma_{\dot x}^2 & 0  & 0 \\
    0 & \sigma_{\dot y y} & 0 & 0 & \sigma_{\dot y}^2  & 0 \\
    0 & 0 & \sigma_{\dot z z} & 0 & 0 & \sigma_{\dot z}^2 \\
   \end{bmatrix} \cdot \sigma_{a}$$
   
with 𝜎𝑎 as the magnitude of the standard deviation of the acceleration, which distrubs the quadcopter. Assume no cross correlation across x,y,z dimensions.

A matrix $G$ can act as an "actor" to help with this.

In [9]:
G = Matrix([t**2/2, t]) # one dimension
display(G*G.T)

⎡ 4   3⎤
⎢T   T ⎥
⎢──  ──⎥
⎢4   2 ⎥
⎢      ⎥
⎢ 3    ⎥
⎢T    2⎥
⎢──  T ⎥
⎣2     ⎦

Get the following by inspection

In [10]:
siga = symbols('\sigma_a') # std dev
sigw = symbols('\sigma_w') # std dev

# Q - Pos and Velocity
Qpv = Matrix([
    [t**4/4,      0,      0, t**3/2,      0,      0],
    [     0, t**4/4,      0,      0, t**3/2,      0],
    [     0,      0, t**4/4,      0,      0, t**3/2],
    [t**3/2,      0,      0,   t**2,      0,      0],
    [     0, t**3/2,      0,      0,   t**2,      0],
    [     0,      0, t**3/2,      0,      0,   t**2],
]) * siga**2

# Q - quaternion
Gq = Matrix(BlockMatrix([
    [zeros(1,3)],
    [eye(3)],
]))
Gq = -t/2*skew4Left(q_toLfromB)*Gq
Qq = Gq*sigw**2*eye(3)*Gq.T

Qq2  = Matrix([
    [1-qw**2,  -qx*qw,  -qy*qw,  -qz*qw],
    [ -qw*qx, 1-qx**2,  -qy*qx,  -qz*qx],
    [ -qw*qy,  -qx*qy, 1-qy**2,  -qz*qy],
    [ -qw*qz,  -qx*qz,  -qy*qz, 1-qz**2]
]) * sigw**2*t**2/4

# Q - Full
Q = Matrix(BlockMatrix([
    [Qpv,        zeros(6,4)],
    [zeros(4,6),         Qq],
]))


print('Q - Pos and Velocity')
display(Qpv, Qpv.shape)
print('Q - Quaternion')
display(Qq, Qq.shape)
print('Q - Full')
display(Q)






Q - Pos and Velocity


⎡ 4        2                             3        2                          ⎤
⎢T ⋅\sigmaₐ                             T ⋅\sigmaₐ                           ⎥
⎢───────────       0            0       ───────────       0            0     ⎥
⎢     4                                      2                               ⎥
⎢                                                                            ⎥
⎢              4        2                             3        2             ⎥
⎢             T ⋅\sigmaₐ                             T ⋅\sigmaₐ              ⎥
⎢     0       ───────────       0            0       ───────────       0     ⎥
⎢                  4                                      2                  ⎥
⎢                                                                            ⎥
⎢                           4        2                             3        2⎥
⎢                          T ⋅\sigmaₐ                             T ⋅\sigmaₐ ⎥
⎢     0            0       ───────────       0      

(6, 6)

Q - Quaternion


⎡ 2         2   2    2         2    2    2         2    2                     
⎢T ⋅\sigma_w ⋅qₓ    T ⋅\sigma_w ⋅q_y    T ⋅\sigma_w ⋅q_z                     -
⎢──────────────── + ───────────────── + ─────────────────                    ─
⎢       4                   4                   4                             
⎢                                                                             
⎢                   2         2                             2         2    2  
⎢                 -T ⋅\sigma_w ⋅q_w⋅qₓ                     T ⋅\sigma_w ⋅q_w   
⎢                 ─────────────────────                    ───────────────── +
⎢                           4                                      4          
⎢                                                                             
⎢                   2         2                                               
⎢                 -T ⋅\sigma_w ⋅q_w⋅q_y                                      -
⎢                 ──────────────────────            

(4, 4)

Q - Full


⎡ 4        2                             3        2                           
⎢T ⋅\sigmaₐ                             T ⋅\sigmaₐ                            
⎢───────────       0            0       ───────────       0            0      
⎢     4                                      2                                
⎢                                                                             
⎢              4        2                             3        2              
⎢             T ⋅\sigmaₐ                             T ⋅\sigmaₐ               
⎢     0       ───────────       0            0       ───────────       0      
⎢                  4                                      2                   
⎢                                                                             
⎢                           4        2                             3        2 
⎢                          T ⋅\sigmaₐ                             T ⋅\sigmaₐ  
⎢     0            0       ───────────       0      

# Predict
## State Transition

In [11]:
X = Matrix([
    r_L,
    v_L,
    q_toLfromB,
    gyroBias,
    accelBias
])

a_B = accelMeas-accelBias
a_L  = qRot(q_toLfromB, a_B) - g_L
a_L2 = Matrix([
    a_L[0]**2,
   a_L[1]**2,
   a_L[2]**2,
])

fx_r = r_L + v_L*t + a_L*t
fx_v = v_L + a_L*t
fx_q = (eye(4) + t/2*skew4Right(wMeas-gyroBias))*q_toLfromB
fx_gB = eye(3) * gyroBias
fx_aB = eye(3) * accelBias


fx = Matrix([
    fx_r,
    fx_v,
    fx_q,
    fx_gB,
    fx_aB
       ])


print('X')
display(X)
print('fx')
display(fx)






X


⎡ rₓ ⎤
⎢    ⎥
⎢r_y ⎥
⎢    ⎥
⎢r_z ⎥
⎢    ⎥
⎢ vₓ ⎥
⎢    ⎥
⎢v_y ⎥
⎢    ⎥
⎢v_z ⎥
⎢    ⎥
⎢q_w ⎥
⎢    ⎥
⎢ qₓ ⎥
⎢    ⎥
⎢q_y ⎥
⎢    ⎥
⎢q_z ⎥
⎢    ⎥
⎢g_bx⎥
⎢    ⎥
⎢g_by⎥
⎢    ⎥
⎢g_bz⎥
⎢    ⎥
⎢a_bx⎥
⎢    ⎥
⎢a_by⎥
⎢    ⎥
⎣a_bz⎦

fx


⎡            ⎛             ⎛   2     2      2      2⎞                         
⎢   T⋅vₓ + T⋅⎝(-a_bx + aₓ)⋅⎝q_w  + qₓ  - q_y  - q_z ⎠ + (-a_by + a_y)⋅(-2⋅q_w⋅
⎢                                                                             
⎢            ⎛                                                    ⎛   2     2 
⎢  T⋅v_y + T⋅⎝(-a_bx + aₓ)⋅(2⋅q_w⋅q_z + 2⋅qₓ⋅q_y) + (-a_by + a_y)⋅⎝q_w  - qₓ  
⎢                                                                             
⎢          ⎛                                                                  
⎢T⋅v_z + T⋅⎝-g + (-a_bx + aₓ)⋅(-2⋅q_w⋅q_y + 2⋅qₓ⋅q_z) + (-a_by + a_y)⋅(2⋅q_w⋅q
⎢                                                                             
⎢         ⎛             ⎛   2     2      2      2⎞                            
⎢       T⋅⎝(-a_bx + aₓ)⋅⎝q_w  + qₓ  - q_y  - q_z ⎠ + (-a_by + a_y)⋅(-2⋅q_w⋅q_z
⎢                                                                             
⎢        ⎛                                          

## State Transition Jacobian

In [12]:
dfdx = fx.jacobian(X)

print('dfdx')
display(dfdx)

dfdx


⎡                                                                             
⎢1  0  0  T  0  0  T⋅(2⋅q_w⋅(-a_bx + aₓ) + 2⋅q_y⋅(-a_bz + a_z) - 2⋅q_z⋅(-a_by 
⎢                                                                             
⎢                                                                             
⎢0  1  0  0  T  0  T⋅(2⋅q_w⋅(-a_by + a_y) - 2⋅qₓ⋅(-a_bz + a_z) + 2⋅q_z⋅(-a_bx 
⎢                                                                             
⎢                                                                             
⎢0  0  1  0  0  T  T⋅(2⋅q_w⋅(-a_bz + a_z) + 2⋅qₓ⋅(-a_by + a_y) - 2⋅q_y⋅(-a_bx 
⎢                                                                             
⎢                                                                             
⎢0  0  0  1  0  0  T⋅(2⋅q_w⋅(-a_bx + aₓ) + 2⋅q_y⋅(-a_bz + a_z) - 2⋅q_z⋅(-a_by 
⎢                                                                             
⎢                                                   

In [13]:
qv = Matrix([q_toLfromB[1:]]).T
QF = Matrix([
    [qv, qw*eye(3)+skew3(qv)]
])
dvdq = 2*QF*Matrix([
                [zeros(1),a_B.T],
                [a_B, -skew3(a_B)]
])

C_toLfromB = quat2dcm(q_toLfromB)
box = Matrix([
    [-qv.T],
    [qw*eye(3) + skew3(qv)]
])


dfdx = Matrix([
    [zeros(3), eye(3), zeros(3,4),zeros(3),zeros(3)],
    [zeros(3), zeros(3), dvdq, zeros(3), -C_toLfromB ],
    [zeros(4,3), zeros(4,3), 1/2*skew4Right(wMeas-gyroBias), -1/2*box, zeros(4,3)],
    [zeros(3),zeros(3),zeros(3,4),zeros(3),zeros(3)],
    [zeros(3),zeros(3),zeros(3,4),zeros(3),zeros(3)]    
])
dfdx = dfdx * t + eye(16)

print('dfdx')
display(dfdx)
mprint(dfdx)
print(dfdx.shape)


dfdx


⎡1  0  0  T  0  0                                  0                          
⎢                                                                             
⎢0  1  0  0  T  0                                  0                          
⎢                                                                             
⎢0  0  1  0  0  T                                  0                          
⎢                                                                             
⎢                                                                             
⎢0  0  0  1  0  0  T⋅(2⋅q_w⋅(-a_bx + aₓ) + 2⋅q_y⋅(-a_bz + a_z) - 2⋅q_z⋅(-a_by 
⎢                                                                             
⎢                                                                             
⎢0  0  0  0  1  0  T⋅(2⋅q_w⋅(-a_by + a_y) - 2⋅qₓ⋅(-a_bz + a_z) + 2⋅q_z⋅(-a_bx 
⎢                                                                             
⎢                                                   

Matrix([
[1, 0, 0, T, 0, 0,                                                                   0,                                                                  0,                                                                   0,                                                                  0,          0,          0,          0,                                      0,                                      0,                                      0],
[0, 1, 0, 0, T, 0,                                                                   0,                                                                  0,                                                                   0,                                                                  0,          0,          0,          0,                                      0,                                      0,                                      0],
[0, 0, 1, 0, 0, T,                                                                   0,    

## Process Noise

In [14]:
Q_r  = (wa*t**2)**2 * eye(3)
Q_v  = (wa*t)**2 * eye(3)
Q_q  = (ww*t/2)**2 *Matrix([
    [1-qw**2,  -qx*qw,  -qy*qw,  -qz*qw],
    [ -qw*qx, 1-qx**2,  -qy*qx,  -qz*qx],
    [ -qw*qy,  -qx*qy, 1-qy**2,  -qz*qy],
    [ -qw*qz,  -qx*qz,  -qy*qz, 1-qz**2]
])
Q_gB = (wgB*t)**2 * eye(3)
Q_aB = (waB*t)**2 * eye(3)

Q = Matrix([
    [Q_r, zeros(3), zeros(3,4), zeros(3), zeros(3)],
    [zeros(3), Q_v, zeros(3,4), zeros(3), zeros(3)],
    [zeros(4,3), zeros(4,3), Q_q, zeros(4,3), zeros(4,3)],
    [zeros(3), zeros(3), zeros(3,4), Q_gB, zeros(3)],
    [zeros(3), zeros(3), zeros(3,4), zeros(3), Q_aB],
    
])

print('Q')
display(Q)
print(Q.shape)


#print('Q_r')
#display(Q_r)
#print('Q_v')
#display(Q_v)
#print('Q_q')
#display(Q_q)
#print('Q_gB')
#display(Q_gB)
#print('Q_aB')
#display(Q_aB)

Q


⎡ 4   2                                                                       
⎢T ⋅wₐ     0       0       0       0       0             0                   0
⎢                                                                             
⎢         4   2                                                               
⎢  0     T ⋅wₐ     0       0       0       0             0                   0
⎢                                                                             
⎢                 4   2                                                       
⎢  0       0     T ⋅wₐ     0       0       0             0                   0
⎢                                                                             
⎢                         2   2                                               
⎢  0       0       0     T ⋅wₐ     0       0             0                   0
⎢                                                                             
⎢                                 2   2             

(16, 16)


# Update

## Measurment Function

In [15]:
hx_accel = -qRot(quatConj(q_toLfromB), g_L)
hx_mag   =  qRot(quatConj(q_toLfromB), B_L)
hx_gps   = r_L
hx = Matrix([
    hx_accel,
    hx_mag,
    hx_gps
])

print('hx')
display(hx)

hx


⎡ -g⋅(-2⋅q_w⋅q_y + 2⋅qₓ⋅q_z)  ⎤
⎢                             ⎥
⎢  -g⋅(2⋅q_w⋅qₓ + 2⋅q_y⋅q_z)  ⎥
⎢                             ⎥
⎢   ⎛   2     2      2      2⎞⎥
⎢-g⋅⎝q_w  - qₓ  - q_y  + q_z ⎠⎥
⎢                             ⎥
⎢   ⎛   2     2      2      2⎞⎥
⎢mₓ⋅⎝q_w  + qₓ  - q_y  - q_z ⎠⎥
⎢                             ⎥
⎢ mₓ⋅(-2⋅q_w⋅q_z + 2⋅qₓ⋅q_y)  ⎥
⎢                             ⎥
⎢  mₓ⋅(2⋅q_w⋅q_y + 2⋅qₓ⋅q_z)  ⎥
⎢                             ⎥
⎢             rₓ              ⎥
⎢                             ⎥
⎢             r_y             ⎥
⎢                             ⎥
⎣             r_z             ⎦

## Measurment Function Jacobian

In [16]:
dhdx = hx.jacobian(X)

print('dhdx')
display(dhdx)
mprint(dhdx)


dhdx


⎡0  0  0  0  0  0   2⋅g⋅q_y   -2⋅g⋅q_z   2⋅g⋅q_w    -2⋅g⋅qₓ   0  0  0  0  0  0
⎢                                                                             
⎢0  0  0  0  0  0   -2⋅g⋅qₓ   -2⋅g⋅q_w  -2⋅g⋅q_z   -2⋅g⋅q_y   0  0  0  0  0  0
⎢                                                                             
⎢0  0  0  0  0  0  -2⋅g⋅q_w    2⋅g⋅qₓ    2⋅g⋅q_y   -2⋅g⋅q_z   0  0  0  0  0  0
⎢                                                                             
⎢0  0  0  0  0  0  2⋅mₓ⋅q_w   2⋅mₓ⋅qₓ   -2⋅mₓ⋅q_y  -2⋅mₓ⋅q_z  0  0  0  0  0  0
⎢                                                                             
⎢0  0  0  0  0  0  -2⋅mₓ⋅q_z  2⋅mₓ⋅q_y   2⋅mₓ⋅qₓ   -2⋅mₓ⋅q_w  0  0  0  0  0  0
⎢                                                                             
⎢0  0  0  0  0  0  2⋅mₓ⋅q_y   2⋅mₓ⋅q_z  2⋅mₓ⋅q_w    2⋅mₓ⋅qₓ   0  0  0  0  0  0
⎢                                                                             
⎢1  0  0  0  0  0      0         0          0       

Matrix([
[0, 0, 0, 0, 0, 0,    2*g*q_y,  -2*g*q_z,    2*g*q_w,   -2*g*q_x, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0,   -2*g*q_x,  -2*g*q_w,   -2*g*q_z,   -2*g*q_y, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0,   -2*g*q_w,   2*g*q_x,    2*g*q_y,   -2*g*q_z, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0,  2*m_x*q_w, 2*m_x*q_x, -2*m_x*q_y, -2*m_x*q_z, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0, -2*m_x*q_z, 2*m_x*q_y,  2*m_x*q_x, -2*m_x*q_w, 0, 0, 0, 0, 0, 0],
[0, 0, 0, 0, 0, 0,  2*m_x*q_y, 2*m_x*q_z,  2*m_x*q_w,  2*m_x*q_x, 0, 0, 0, 0, 0, 0],
[1, 0, 0, 0, 0, 0,          0,         0,          0,          0, 0, 0, 0, 0, 0, 0],
[0, 1, 0, 0, 0, 0,          0,         0,          0,          0, 0, 0, 0, 0, 0, 0],
[0, 0, 1, 0, 0, 0,          0,         0,          0,          0, 0, 0, 0, 0, 0, 0]])
